In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import FMClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.feature import MinMaxScaler, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# $example off$
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession \
    .builder \
    .appName("FMClassifierExample") \
    .getOrCreate()

# $example on$
# Load and parse the data file, converting it to a DataFrame.


data = spark.read.option("delimiter", "\t").csv("./movie_sample.txt",inferSchema="true")

In [3]:
data.show()

+---+---+---+--------------------+---+---+---+---+
|_c0|_c1|_c2|                 _c3|_c4|_c5|_c6|_c7|
+---+---+---+--------------------+---+---+---+---+
|  1|  1|  1|186,0,0,0,0,0,0,0...|  1|112|  2|  1|
|  1|  1|  1|186,0,0,0,0,0,0,0...|  1| 38|  5|  0|
|  1|  1|  1|186,0,0,0,0,0,0,0...|  1|151|  7|  0|
|  1|  1|  1|186,0,0,0,0,0,0,0...|  1| 77|  6|  0|
|  1|  1|  1|186,0,0,0,0,0,0,0...|  1|188|  9|  0|
|  1|  1|  1|186,0,0,0,0,0,0,0...|  1|164|  9|  0|
|  1|  1|  1|112,186,0,0,0,0,0...|  2| 84|  7|  1|
|  1|  1|  1|112,186,0,0,0,0,0...|  2| 76|  8|  0|
|  1|  1|  1|112,186,0,0,0,0,0...|  2|162|  6|  0|
|  1|  1|  1|112,186,0,0,0,0,0...|  2| 94|  1|  0|
|  1|  1|  1|112,186,0,0,0,0,0...|  2| 73|  5|  0|
|  1|  1|  1|112,186,0,0,0,0,0...|  2| 49|  7|  0|
|  1|  1|  1|84,112,186,0,0,0,...|  3| 52|  4|  1|
|  1|  1|  1|84,112,186,0,0,0,...|  3|205|  3|  0|
|  1|  1|  1|84,112,186,0,0,0,...|  3|100|  2|  0|
|  1|  1|  1|84,112,186,0,0,0,...|  3|141|  9|  0|
|  1|  1|  1|84,112,186,0,0,0,.

In [4]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)



In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
features_cols =['_c0', '_c1', '_c2', '_c4', '_c5', '_c6']


print(features_cols)

['_c0', '_c1', '_c2', '_c4', '_c5', '_c6']


In [7]:
# 组合特征到向量
assembler = VectorAssembler(
    inputCols=features_cols,
    outputCol="raw_features")

In [8]:
data_df2 = assembler.transform(data)

In [9]:
data_df2.printSchema()
data_df2.show(5, False)

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- raw_features: vector (nullable = true)

+---+---+---+-----------------------------------------------------------------------------------------------------+---+---+---+---+---------------------------+
|_c0|_c1|_c2|_c3                                                                                                  |_c4|_c5|_c6|_c7|raw_features               |
+---+---+---+-----------------------------------------------------------------------------------------------------+---+---+---+---+---------------------------+
|1  |1  |1  |186,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0|1  |112|2  |1  |[1.0,1.0,1.0,1.0,112.0,2.0]|
|1  |1  |1  |186,0,0,0,

In [14]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="_c7", outputCol="label").fit(data)
# Scale features.
featureScaler = MinMaxScaler(inputCol="raw_features", outputCol="features").fit(data_df2)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data_df2.randomSplit([0.7, 0.3])

layers = [4, 5, 4, 2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)


pipeline = Pipeline(stages=[labelIndexer, featureScaler, trainer])


model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
# # compute accuracy on the test set
# result = model.transform(testData)
# predictionAndLabels = result.select("prediction", "label")
# evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
# print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

In [16]:
predictions.show()

Py4JJavaError: An error occurred while calling o701.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 290.0 failed 1 times, most recent failure: Lost task 0.0 in stage 290.0 (TID 283) (192.168.3.34 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (ProbabilisticClassificationModel$$Lambda$3637/1305162264: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_subExpr_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodeg